In [116]:

%matplotlib widget
#%matplotlib notebook
### ---> General
import numpy as np
from scipy import interpolate

### ---> Cosmology
from astropy import constants as const
from astropy.cosmology import Planck15
from astropy import units as u


from colossus.cosmology import cosmology
from colossus.lss import mass_function
cosmo = cosmology.setCosmology('planck18')


### ---> plotting
from matplotlib import pyplot as plt

import matplotlib.colors as mcolors
import matplotlib.cm as cm
from matplotlib.colors import ListedColormap


plt.rc('xtick',labelsize=16)
plt.rc('ytick',labelsize=16)
plt.rc('mathtext', fontset='stix')
plt.rc('font', family='STIXGeneral')
plt.rc('font', size=15)
plt.rc('figure', autolayout=True)
plt.rc('axes', titlesize=16, labelsize=17)
plt.rc('lines', linewidth=2, markersize=6)
plt.rc('legend', fontsize=15)
plt.rc('figure')



In [117]:
class halo:
    def __init__(self, M_halo, R_ns_over_R_NFW):
        self.M_halo = M_halo
        self.R_ns_over_R_NFW = R_ns_over_R_NFW

        self.C_lst = self.concentration()

        ### First we compute the normalized masses of the haloes then obtain rho_NFW_0/rho_crit
        ### NFW_mass_integral stands for 4 pi Int dx x^2 NFW(x)
        NFW_mass_integral = self.NFW_tilde_integral_1(self.C_lst) - self.NFW_tilde_integral_1(np.zeros(len(self.C_lst)))
        rho_NFW_0_over_rho_crit = 200.*self.C_lst**3/3./NFW_mass_integral


        ### Now we can obtain rho_NFW_0 for all the halos
        rho_crit = Planck15.critical_density(0).to(u.Msun/u.kpc**3) # M_sun/kpc^3
        self.rho_NFW_0_lst = rho_NFW_0_over_rho_crit*rho_crit # M_sun/kpc^3


        ### Similarly, we obtain the R_NFW_0 and R_vir for all the halos
        R_NFW_0_cube_rho_crit = self.M_halo/NFW_mass_integral/4./np.pi/rho_NFW_0_over_rho_crit
        self.R_NFW_0_lst = (R_NFW_0_cube_rho_crit/rho_crit)**(1./3.) # kpc
        self.R_vir_lst = self.R_NFW_0_lst*self.C_lst # kpc


        ### Number of neutron stars in each halo is computed as
        SFHR_params = [10**(-1.777), 10**(11.514), -1.412, 0.316, 3.508]
        self.M_star_lst = self.stellar_mass(SFHR_params)
        self.N_ns_lst = self.N_ns(SFHR_params)["N_ns"]
        self.R_ns_0_lst = R_ns_over_R_NFW*self.R_NFW_0_lst

        ### We evaluate rho_ns_0 by the normalization condition 4 pi rho_ns_0 R_ns_0^3 Int dx x^2 NS(x) = N_ns M_ns
        ### M_ns is assumed to be 1 M_sun
        ns_mass_integral = self.NS_tilde_integral(self.C_lst*self.R_NFW_0_lst/self.R_ns_0_lst) - self.NS_tilde_integral(0.)
        denominator = 4.*np.pi*self.R_ns_0_lst**3*ns_mass_integral
        numerator = self.N_ns_lst*u.M_sun
        self.rho_ns_0_lst = numerator/denominator


        ### We also prepare the NFW and ns profiles for later integrals
        self.x_lst = np.geomspace(1e-5, self.C_lst, 10000).T
        self.NFW_profiles = self.NFW_tilde(self.x_lst)
        self.ns_profiles = self.NS_tilde(self.x_lst, np.array([self.R_NFW_0_lst, self.R_ns_0_lst]))
        
        
        ### The virial velocities and velocity dispersion of each halo
        C_max = 2.1626
        self.v_vir_lst = np.sqrt(const.G*self.M_halo/self.R_vir_lst).to(u.km/u.s) # km/s
        self.v_dm_lst = self.v_vir_lst*np.sqrt(self.C_lst/self.g(self.C_lst))\
                        /np.sqrt(C_max/self.g(C_max))/np.sqrt(2.) # km/s


    def NFW_tilde(self, x):
            
            return 1./x/(1. + x)**2
    
    def NFW_tilde_integral_1(self, x):
        ### Int x^2 NFW(x) dx
        
        return 1./(1. + x) + np.log(1. + x)

    def NFW_tilde_integral_2(self, x):
        ### Int x^2 NFW(x)^2 dx
        
        return -1./3./(1. + x)**3

    def NS_tilde_integral(self, x):
        ### Int x^2 NS(x) dx
        
        return (-2. -2.*x - x**2)*np.exp(-x)
        #return np.exp(-x)*(-6. - 6.*x - 3.*x**2 - x**3)


    def NS_tilde(self, x, params):
        
        params_length = len(params[0,:])
        
        R_NFW_0 = params[0,:].reshape(params_length, 1)
        R_ns_0 = params[1,:].reshape(params_length, 1)

        return np.exp(-x*R_NFW_0/R_ns_0)

    def N_ns(self, SHMR_params):
        
        salpeter_prefac = 0.06 # If int phi(m)dm = 1, then salpeter_prefac = 0.06
        salpeter_power = -2.35


        NS_M_min = 8. # M_sun
        NS_M_max = 100. # M_sun 

        M_stellar = self.stellar_mass(SHMR_params)
        #M_NGC_4993 = 2.95*1e10 M_sun

        integral_1 = salpeter_prefac*(1./(salpeter_power + 1.))*\
                            (NS_M_max**(salpeter_power + 1.) - NS_M_min**(salpeter_power + 1.))
        
        
        t_0 = 3. #Gyr
        tau = 0.1

        t_life = 0.02 # Gyr
        t_lst = np.linspace(0.1 - t_life, 10 - t_life, 100)
        psi = self.SFH(t_lst, [t_0, tau])

        integral_2 = np.trapz(psi, t_lst)

        N_ns = M_stellar*integral_1*integral_2
        return {"N_ns":N_ns, "M_stellar":M_stellar, "SFH":psi}


    def SFH(self, t_lst, SFH_params):
        t_0, tau = SFH_params

        tmp_1 = 1./t_lst/np.sqrt(2.*np.pi*tau**2)
        tmp_2 = np.exp(-(np.log(t_lst) - np.log(t_0))**2/2./tau**2)
        
        return tmp_1*tmp_2


    def g(self, C):
        return np.log(1. + C) - C/(1. + C)



    def stellar_mass(self, SHMR_params):
        ### --- Fitting function from appendix A of 1401.7329
        ### --- M_halo in M_sun

        epsilon, M_1, alpha, gamma, delta = SHMR_params
        f = lambda x: -np.log10(10**(alpha*x) + 1.) + delta*(np.log10(1. + np.exp(x))**gamma)/(1. + np.exp(10**(-x)))
        
        M_star = epsilon*M_1*10**(f(np.log10(self.M_halo.value/M_1)) - f(0.))
        #print("M_star = ", M_star)

        
        return M_star

    def concentration(self):
        ### --- Fitting function from appendix C of 1601.02624
        ### --- M_halo in M_sun/h

        h = 0.7
        reference_mass = 1e10*h*u.Msun # in M_sun/h
        xi = 1./(self.M_halo/reference_mass)

        c_0 = 3.395
        beta = 0.307
        gamma_1 = 0.628
        gamma_2 = 0.317
        nu_0 = 4.135 - 0.564 - 0.210 + 0.0557 - 0.00348

        delta_sc = 1.686
        sigma_M = 22.26*xi**0.292/(1. + 1.53*xi**0.275 + 3.36*xi**0.198)
        nu = delta_sc/sigma_M

        concentration = c_0*(nu/nu_0)**(-gamma_1)*(1. + (nu/nu_0)**(1./beta))**(-beta*(gamma_2 - gamma_1))

        return concentration

    def halo_properties(self):

        halo_property_dictionary = {"Concentration":self.C_lst, "rho_NFW_0":self.rho_NFW_0_lst, "R_NFW_0":self.R_NFW_0_lst, "R_virial":self.R_vir_lst, \
                            "N_ns":self.N_ns_lst, "rho_ns_0":self.rho_ns_0_lst,
                            "x_lst":self.x_lst, "NFW_profiles":self.NFW_profiles, "ns_profiles":self.ns_profiles, "M_star":self.M_star_lst}

        return halo_property_dictionary
        




    


class rates:
    def __init__(self, halo, binary_mass_params):
        self.halo = halo
        self.m_1 = binary_mass_params["m_1"]
        self.m_2 = binary_mass_params["m_2"]
        self.M = binary_mass_params["M"]
        self.eta = binary_mass_params["eta"]
        

    def cross_section(self):

        ### Now we turn to velocities
        
        v_vir_lst = self.halo.v_vir_lst
        v_dm_lst = self.halo.v_dm_lst


        ### Here is our Maxwell-Boltzmann distribution for each halo
        v_PBH_lst = np.linspace(1e-12, np.ones(len(v_vir_lst)), 10000).T ### Normalized to v_vir

        exponent_1_numerator = v_PBH_lst**2*v_vir_lst.reshape(len(v_vir_lst), 1)**2
        exponent_1_denominator = v_dm_lst.reshape(len(v_dm_lst), 1)**2
        exponent_1 = -exponent_1_numerator/exponent_1_denominator

        exponent_2_numerator = np.ones(np.shape(v_PBH_lst))*v_vir_lst.reshape(len(v_vir_lst), 1)**2
        exponent_2_denominator = v_dm_lst.reshape(len(v_dm_lst), 1)**2
        exponent_2 = -exponent_2_numerator/exponent_2_denominator

        ### The MB dist
        P_v_PBH = np.exp(exponent_1) - np.exp(exponent_2)

        ### We normalize the distributions
        normalization_integral = v_vir_lst**3*np.trapz(P_v_PBH*v_PBH_lst**2, v_PBH_lst, axis = 1)
        F_0_lst = 1./4./np.pi/normalization_integral

        ### The normalized distributions
        P_v_PBH = F_0_lst.reshape(len(F_0_lst), 1)*P_v_PBH

        ### The expectation value of the velocity term
        #4.*np.pi*
        # In units of (km/s)^-11/7
        avg_vel_PBH_factor = v_vir_lst**(10./7.)*np.trapz(P_v_PBH*v_PBH_lst**(3./7.), v_PBH_lst, axis = 1)
        # In units of (kpc/yr)^-11/7
        self.avg_vel_PBH_factor = avg_vel_PBH_factor.to(u.kpc**(-11./7.)/u.year**(-11./7.)) 




        ### The velocity independent part of <sigma v>
        sigma_prefac = np.pi*(85.*np.pi/3.)**(2./7.)*2.**(4./7.)*const.G**2*M**2*eta**(2./7.)*const.c**(-10./7.)
        sigma_prefac = sigma_prefac.to(u.kpc**(32/7)/u.year**(18/7))

        ### <sigma v> in units of kpc^3/yr
        self.sigma_v_avg = sigma_prefac*avg_vel_PBH_factor

        ### sigma for a reference velocity of 200 km/s, in units of kpc^2
        reference_vel = (200.*u.km/u.s).to(u.kpc/u.year)
        sigma_reference_200 = sigma_prefac*reference_vel**(-18./7.)

        velocity_dictionary = {"v_vir_lst":v_vir_lst, 
                               "v_dm_lst":v_dm_lst, 
                               "sigma_v_avg":self.sigma_v_avg, 
                               "sigma_reference_200":sigma_reference_200}

        return velocity_dictionary

    def rates_per_halo(self):

        cross_section_properties = self.cross_section()

        ### PBH-PBH merger rate per halo
        
        ### NFW-NFW integral
        density_integral = np.trapz(self.halo.x_lst**2*self.halo.NFW_profiles**2, self.halo.x_lst, axis = 1)
        ### PBH-PBH rate per halo in units of 1/yr
        PBHPBH_Rate_per_halo = self.sigma_v_avg*self.halo.rho_NFW_0_lst**2*self.halo.R_NFW_0_lst**3*0.5*4.*np.pi*density_integral/self.m_1/self.m_2
        self.PBHPBH_Rate_per_halo = PBHPBH_Rate_per_halo.to(1./u.year)

        ### Analytical expression
        ### The C-dependent part
        tmp_C_term = (1. - 1./(1. + self.halo.C_lst)**3)/self.halo.g(C_lst)**2

        ### Other Halo dependent parts
        tmp_halo_term = self.halo.M_halo**2*self.avg_vel_PBH_factor/self.halo.R_NFW_0_lst**3

        ### The numerical prefactor
        anal_prefac = (85.*np.pi/3.)**(2./7.)*const.G**2*const.c**(-10./7.)/6.
        #(85.*np.pi/12./np.sqrt(2.))**(2./7.)*9.*2.**(3./7.)*const.G**2*const.c**(-10./7.)

        ### PBH-PBH rate per halo
        PBHPBH_Rate_per_halo_anal = anal_prefac*tmp_C_term*tmp_halo_term
        ### In units of 1/yr
        self.PBHPBH_Rate_per_halo_anal = PBHPBH_Rate_per_halo_anal.to(1./u.year)

        ### PBH-NS merger rate per halo
 
        ### NFW-NS integral
        density_integral = np.trapz(self.halo.x_lst**2*self.halo.NFW_profiles*self.halo.ns_profiles, self.halo.x_lst, axis = 1)
        ### PBH-PBH rate per halo in units of 1/yr
        PBHNS_Rate_per_halo = self.sigma_v_avg*self.halo.rho_NFW_0_lst*self.halo.rho_ns_0_lst*\
                                            self.halo.R_NFW_0_lst**3*4.*np.pi*density_integral/self.m_1/self.m_2
        self.PBHNS_Rate_per_halo = PBHNS_Rate_per_halo.to(1./u.year)


        rate_per_halo_dictionary = {"PBHPBH_Rate_per_halo":self.PBHPBH_Rate_per_halo, 
                                    "PBHPBH_Rate_per_halo_anal":self.PBHPBH_Rate_per_halo_anal, 
                                    "PBHNS_Rate_per_halo":self.PBHNS_Rate_per_halo}

        return rate_per_halo_dictionary
        

    def integrated_rates(self, M_c_lst):

        rates_per_halo = self.rates_per_halo()

        PBHPBH_rate_lst = np.zeros(len(M_c_lst))
        PBHNS_rate_lst = np.zeros(len(M_c_lst))

        halo_mass_function = mass_function.massFunction(self.halo.M_halo.value, 0., mdef = '200c', model = 'tinker08', q_out = 'dndlnM')

        for indx, val in enumerate(M_c_lst):
            PBHPBH_rate_y = ((1./0.7**3)*self.PBHPBH_Rate_per_halo.value*1e9*halo_mass_function)[self.halo.M_halo.value > val]
            PBHPBH_rate_x = (np.log(self.halo.M_halo.value))[self.halo.M_halo.value > val]
    
            PBHNS_rate_y = ((1./0.7**3)*self.PBHNS_Rate_per_halo.value*1e9*halo_mass_function)[self.halo.M_halo.value > val]
            PBHNS_rate_x = (np.log(self.halo.M_halo.value))[self.halo.M_halo.value > val]

            PBHPBH_rate_lst[indx] = np.trapz(PBHPBH_rate_y, PBHPBH_rate_x)
            PBHNS_rate_lst[indx] = np.trapz(PBHNS_rate_y, PBHNS_rate_x)

        return {"halo_mass_function":halo_mass_function, "PBHPBH_rate_lst":PBHPBH_rate_lst, "PBHNS_rate_lst":PBHNS_rate_lst}

        

    
class spike:
    
    def __init__(self, halo, gamma):
        self.halo = halo


        a = 8.12
        b = 4.24

        alpha_gamma_interp = np.array([0.00733, 0.120, 0.140, 0.142, 0.135, 0.122, 0.103, 0.0818, 0.017])
        gamma_interp = np.array([0.05, 0.2, 0.4,0.6, 0.8, 1.0, 1.2, 1.4, 2.])
        alpha_gamma_interpolation_function = interpolate.interp1d(gamma_interp, alpha_gamma_interp)
        
        if gamma <= min(gamma_interp) or gamma > max(gamma_interp):
            print("ERROR gamma = %5.3f is not supported." % gamma)
        else:
            self.alpha_gamma = alpha_gamma_interpolation_function(gamma)
        
        reference_vel = 200*u.km/u.s
        self.M_SMBH = (10**a*(self.halo.v_dm_lst/reference_vel)**b)*u.Msun
    
        self.R_schw = 2.*const.G*self.M_SMBH/const.c**2
        self.R_spike = self.alpha_gamma*self.halo.R_NFW_0_lst*(self.M_SMBH/self.halo.rho_NFW_0_lst/self.halo.R_NFW_0_lst**3)**(1./(3. - gamma))
        
        self.gamma_spike = (9. - 2.*gamma)/(4. - gamma)
        self.rho_spike_0 = self.halo.rho_NFW_0_lst*(self.R_spike/self.halo.R_NFW_0_lst)**(-gamma)
    


    def spike_properties(self):
        
        spike_dictionary = {"M_SMBH":self.M_SMBH,
                            "R_spike":self.R_spike, 
                            "R_schw":self.R_schw, 
                            "rho_spike_0":self.rho_spike_0, 
                            "alpha_gamma":self.alpha_gamma}
        
        return spike_dictionary

    def spike_profiles(self):
        '''
            Returns the spike profiles without the rho_spike_0 prefactor and for all halos 
        '''
        ratio = (self.R_schw/self.R_spike).to(u.m**0)
        length = len(ratio)

        x_lst = np.geomspace(4.*ratio, np.ones(length), 10000).T
        print(np.shape(x_lst))

        spike_profiles = (1. - 4.*ratio.reshape(length, 1)/x_lst)**3*(1./x_lst)**self.gamma_spike

        return {"x_list":x_lst, "spike_profiles":spike_profiles}



In [118]:
M_halo_lst = np.geomspace(1e3, 1e15, 1000)*(0.7)*u.Msun # Halo masses in M_sun/h units

m_1 = 30.*u.Msun
m_2 = 30.*u.Msun
M = m_1 + m_2
eta = m_1*m_2/M**2
binary_mass_params = {"m_1":m_1, "m_2":m_2, "M":M, "eta":eta}

R_ns_over_R_NFW = 0.1

NFW_halo = halo(M_halo_lst, R_ns_over_R_NFW)
merger_rates = rates(NFW_halo, binary_mass_params)




halo_properties = NFW_halo.halo_properties()

print("Halo propoeries: ", halo_properties.keys())

C_lst = halo_properties["Concentration"]
R_NFW_0_lst = halo_properties["R_NFW_0"]
R_vir_lst = halo_properties["R_virial"]
N_ns_lst = halo_properties["N_ns"]
#print(N_ns_lst)
NFW_profiles = halo_properties["NFW_profiles"]
ns_profiles = halo_properties["ns_profiles"]
x_lst = halo_properties["x_lst"]
rho_NFW_0_lst = halo_properties["rho_NFW_0"]
rho_ns_0_lst = halo_properties["rho_ns_0"]
M_star_lst = halo_properties["M_star"]

R_ns_0_lst = R_ns_over_R_NFW*R_NFW_0_lst


cross_section_properties = merger_rates.cross_section()
print("Cross-section properties: ", cross_section_properties.keys())
v_vir_lst = cross_section_properties["v_vir_lst"]
v_dm_lst = cross_section_properties["v_dm_lst"] 
sigma_v_avg = cross_section_properties["sigma_v_avg"]
sigma_reference_200 = cross_section_properties["sigma_reference_200"]


rates_per_halo = merger_rates.rates_per_halo()
print("Per-halo rates: ", rates_per_halo.keys())
PBHPBH_Rate_per_halo = rates_per_halo["PBHPBH_Rate_per_halo"]
PBHPBH_Rate_per_halo_anal = rates_per_halo["PBHPBH_Rate_per_halo_anal"]
PBHNS_Rate_per_halo = rates_per_halo["PBHNS_Rate_per_halo"]


M_c_lst = np.geomspace(1e3, 1e14, 10) 
integarted_rates = merger_rates.integrated_rates(M_c_lst)
print("Integrated rates: ", integarted_rates.keys())
halo_mass_function = integarted_rates["halo_mass_function"]
PBHPBH_rate_lst = integarted_rates["PBHPBH_rate_lst"]
PBHNS_rate_lst = integarted_rates["PBHNS_rate_lst"]






/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:130: RuntimeWarning: overflow encountered in exp


Halo propoeries:  dict_keys(['Concentration', 'rho_NFW_0', 'R_NFW_0', 'R_virial', 'N_ns', 'rho_ns_0', 'x_lst', 'NFW_profiles', 'ns_profiles', 'M_star'])
Cross-section properties:  dict_keys(['v_vir_lst', 'v_dm_lst', 'sigma_v_avg', 'sigma_reference_200'])
Per-halo rates:  dict_keys(['PBHPBH_Rate_per_halo', 'PBHPBH_Rate_per_halo_anal', 'PBHNS_Rate_per_halo'])
Integrated rates:  dict_keys(['halo_mass_function', 'PBHPBH_rate_lst', 'PBHNS_rate_lst'])


In [119]:
#C_lst_analytical = 4.67*(M_halo_lst/1e14)**(-0.11)


fig = plt.figure()
ax = fig.gca()

ax.plot(M_halo_lst, C_lst, c = "k")
#ax.plot(M_halo_lst, C_lst_analytical)

ax.set_xscale("log")
ax.set_yscale("log")

ax.set_xlabel(r"$M_\mathrm{h}[M_\mathrm{Sun}]$")
ax.set_ylabel(r"$C[M_\mathrm{h}]$")

ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])


fig = plt.figure()
ax = fig.gca()

ax.plot(M_halo_lst, R_NFW_0_lst, label = r"$R_\mathrm{NFW}^0 [\mathrm{kpc}]$")
ax.plot(M_halo_lst, R_vir_lst, label = r"$R_\mathrm{vir} [\mathrm{kpc}]$")

ax.plot(M_halo_lst, R_ns_0_lst, label = r"$R_\mathrm{ns}^0 [\mathrm{kpc}]$")
ax.plot(M_halo_lst, N_ns_lst, label = r"$N_\mathrm{ns}$")

ax.plot(M_halo_lst, rho_NFW_0_lst, label = r"$\rho^{(0)}_\mathrm{NFW}$")
ax.plot(M_halo_lst, rho_ns_0_lst, label = r"$\rho^{(0)}_\mathrm{ns}$")


ax.set_xscale("log")
ax.set_yscale("log")


ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])
ax.legend()

ax.set_xlabel(r"$M_\mathrm{h}[M_\mathrm{Sun}]$")



fig = plt.figure()
ax = fig.gca()

for indx, x_lst_tmp in enumerate(x_lst):
    if indx%40 == 0:
        ax.plot(x_lst_tmp, rho_NFW_0_lst[indx]*NFW_profiles[indx], c = "k", alpha = 1. - indx/len(x_lst))
        
        ax.plot(x_lst_tmp, rho_ns_0_lst[indx]*ns_profiles[indx], c = "darkred", alpha = 1. - indx/len(x_lst))



ax.set_xscale("log")
ax.set_yscale("log")

ax.set_xlim([np.min(x_lst), np.max(x_lst)])
ax.set_ylim([1e-4, 1e13])


ax.set_xlabel(r"$r/R_\mathrm{NFW}^0$")
ax.set_ylabel(r"$\rho [M_\mathrm{Sun}/\mathrm{kpc}^3]$")

FigureCanvasNbAgg()

FigureCanvasNbAgg()

FigureCanvasNbAgg()

Text(0, 0.5, '$\\rho [M_\\mathrm{Sun}/\\mathrm{kpc}^3]$')

In [120]:
gamma = 2.
DM_spike = spike(NFW_halo, gamma)

spike_properties = DM_spike.spike_properties()
print("Spike properties: ", spike_properties.keys())
M_SMBH_lst = spike_properties["M_SMBH"].to(u.M_sun)
rho_spike_0_lst = spike_properties["rho_spike_0"].to(u.M_sun/u.kpc**3)
R_spike_0_lst = spike_properties["R_spike"].to(u.pc)


spike_profiles = DM_spike.spike_profiles()
x_lst_spike = spike_profiles["x_list"]
r_lst_spike = spike_profiles["x_list"]*R_spike_0_lst.reshape(len(R_spike_0_lst), 1)
spike_profiles = spike_profiles["spike_profiles"]

fig = plt.figure()
ax = fig.gca()

for indx, r_lst_spike_tmp in enumerate(r_lst_spike):
    if indx%40 == 0:
        ax.plot(r_lst_spike_tmp, rho_spike_0_lst[indx]*spike_profiles[indx], c = "k", alpha = 1. - indx/len(r_lst_spike))


ax.set_xscale("log")
ax.set_yscale("log")

#ax.set_xlim([np.min(x_lst_spike), np.max(x_lst_spike)])
ax.set_ylim([1e10, 1e55])

ax.set_xlabel(r"$r [\mathrm{pc}]$")
ax.set_ylabel(r"$\rho_\mathrm{spike} [M_\mathrm{Sun}/\mathrm{kpc}^3]$")

Spike properties:  dict_keys(['M_SMBH', 'R_spike', 'R_schw', 'rho_spike_0', 'alpha_gamma'])
(1000, 10000)


FigureCanvasNbAgg()

Text(0, 0.5, '$\\rho_\\mathrm{spike} [M_\\mathrm{Sun}/\\mathrm{kpc}^3]$')

In [121]:
M_pbh = 1.*u.M_sun


ref_velo = 200*u.km/u.s
velo_prefac_tmp = (const.G*M_SMBH_lst/R_spike_0_lst)**(1./2.)/ref_velo
velo_prefac = velo_prefac_tmp**(-11./7.)*ref_velo

sigma_v_prefac = 1.4*1e-14*(M_pbh/30./u.M_sun)**2*velo_prefac*u.pc**2


### PBH-PBH
integral = np.trapz(spike_profiles**2*x_lst_spike**2*x_lst_spike**(11./14.), x_lst_spike, axis = 1)



radius_prefac = 4.*np.pi*0.5*R_spike_0_lst**3*rho_spike_0_lst**2/M_pbh**2

integral_prefac = (sigma_v_prefac*radius_prefac).to(1./u.yr)
N_spike_PBH_PBH = integral_prefac*integral

### PBH-NS
ns_profiles = NFW_halo.NS_tilde(x_lst_spike, np.array([R_spike_0_lst, R_ns_0_lst]))
integral = np.trapz(spike_profiles*ns_profiles*x_lst_spike**2*x_lst_spike**(11./14.), x_lst_spike, axis = 1)

radius_prefac = 4.*np.pi*0.5*R_spike_0_lst**3*rho_spike_0_lst*rho_ns_0_lst/M_pbh**2

integral_prefac = (sigma_v_prefac*radius_prefac).to(1./u.yr)
N_spike_PBH_NS = integral_prefac*integral




fig = plt.figure()
ax = fig.gca()

ax.plot(M_SMBH_lst.value, N_spike_PBH_PBH, label = r"PBH-PBH, $f_\mathrm{PBH} = 1$", c = "k", ls = "--")
ax.plot(M_SMBH_lst.value, 1e-6*N_spike_PBH_PBH, label = r"PBH-PBH, $f_\mathrm{PBH} = 10^{-3}$", c = "k")

ax.plot(M_SMBH_lst.value, N_spike_PBH_NS, label = r"PBH-NS, $f_\mathrm{PBH} = 1$", c = "darkred", ls = "--")
ax.plot(M_SMBH_lst.value, 1e-3*N_spike_PBH_NS, label = r"PBH-NS, $f_\mathrm{PBH} = 10^{-3}$", c = "darkred")



ax.set_xscale("log")
ax.set_yscale("log")

ax.legend(ncol = 1, loc = 4)

ax.set_xlim([1e6, 1e8])
#ax.set_ylim([1e-16, 1e-11])

ax.set_xlabel(r"$M_\mathrm{SMBH}[M_\mathrm{Sun}]$")
ax.set_ylabel(r"Merger rate per spike $N_\mathrm{sp}[1/\mathrm{yr}]$")

plt.title(r"$\gamma = 2$")

ax.axhline([1e-2], c = "gray", ls = "--")
ax.axhline([1e-4], c = "gray", ls = "--")



FigureCanvasNbAgg()

In [122]:
print(max(R_spike_0_lst/R_NFW_0_lst))
print(max(R_spike_0_lst/R_ns_0_lst))

0.020482784536551942 pc / kpc
0.20482784536551943 pc / kpc


In [123]:
fig = plt.figure()
ax = fig.gca()


ax.plot(M_halo_lst, M_star_lst/M_halo_lst, c = "black", ls = "-")

ax.plot(M_halo_lst, 1e-2*M_halo_lst/M_halo_lst, c = "gray", ls = "--")



ax.set_xscale("log")
ax.set_yscale("log")

ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])

ax.set_ylabel(r"$M_\mathrm{star}[M_\mathrm{Sun}/h]/M_\mathrm{h}[M_\mathrm{Sun}/h]$")
ax.set_xlabel(r"$M_\mathrm{h}[M_\mathrm{Sun}/h]$")

FigureCanvasNbAgg()

Text(0.5, 0, '$M_\\mathrm{h}[M_\\mathrm{Sun}/h]$')

In [124]:
sigma_analytical = 100*(M_halo_lst/1e12)**(1./3.)


print("sigma at v = 200 km/s: ", sigma_reference_200)


fig = plt.figure()
ax = fig.gca()

ax.plot(M_halo_lst, v_vir_lst, label = r"$v_\mathrm{vir}$", c = "k")
ax.plot(M_halo_lst, v_dm_lst, label = r"$v_\mathrm{dm}$", c = "red")
ax.plot(M_halo_lst, sigma_analytical, label = r"$v_\mathrm{dm - anal}$", c = "lightblue")


ax.set_xscale("log")
ax.set_yscale("log")
ax.legend()

ax.set_xlabel(r"$M_\mathrm{h}[M_\mathrm{Sun}/h]$")
ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])


sigma at v = 200 km/s:  1.369606200140508e-20 kpc2


FigureCanvasNbAgg()

(700.0, 700000000000000.0)

In [125]:





fig = plt.figure()
ax = fig.gca()

ax.plot(M_halo_lst, PBHPBH_Rate_per_halo, label = r"PBH-PBH, $f_\mathrm{PBH} = 1$", c = "k", alpha = 1)
ax.plot(M_halo_lst, 1e-6*PBHPBH_Rate_per_halo, label = r"PBH-PBH, $f_\mathrm{PBH} = 1e-3$", c = "k", alpha = 0.4)


ax.plot(M_halo_lst, PBHNS_Rate_per_halo, label = r"PBH-NS, $f_\mathrm{PBH} = 1$", c = "red", alpha = 1)
ax.plot(M_halo_lst, 1e-3*PBHNS_Rate_per_halo, label = r"PBH-NS, $f_\mathrm{PBH} = 1e-3$", c = "red", alpha = 0.4)


ax.set_xscale("log")
ax.set_yscale("log")

ax.legend(ncol = 2)

ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])
#ax.set_ylim([1e-16, 1e-11])

ax.set_xlabel(r"$M_\mathrm{h}[M_\mathrm{Sun}/h]$")
ax.set_ylabel(r"Merger rate per halo $\mathcal{R}[1/\mathrm{yr}]$")



fig = plt.figure()
ax = fig.gca()


#ax.plot(M_halo_lst.value*0.7, PBHPBH_Rate_per_halo.value/PBHPBH_Rate_per_halo_anal.value)
ax.plot(M_halo_lst.value, PBHPBH_Rate_per_halo)

ax.plot(M_halo_lst, PBHPBH_Rate_per_halo_anal, ls = "--")

#

ax.set_xscale("log")
ax.set_yscale("log")



ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])
#ax.set_ylim([1e-16, 1e-11])


ax.set_xlabel(r"$M_\mathrm{h}[M_\mathrm{Sun}/h]$")
ax.set_ylabel(r"$\mathcal{R}[1/\mathrm{yr}]$")



FigureCanvasNbAgg()

FigureCanvasNbAgg()

Text(0, 0.5, '$\\mathcal{R}[1/\\mathrm{yr}]$')

In [126]:
fig = plt.figure()
ax = fig.gca()

ax.scatter(M_c_lst, PBHPBH_rate_lst, c = "k")
ax.scatter(M_c_lst, 1e1*PBHNS_rate_lst, c = "red")

ax.plot(M_c_lst, PBHPBH_rate_lst, label = "PBH-PBH", c = "k")
ax.plot(M_c_lst, 1e1*PBHNS_rate_lst, label = "PBH-NS", c = "red")


ax.set_xscale("log")
ax.set_yscale("log")


ax.legend()


ax.set_xlabel(r"$M_\mathrm{c}[M_\mathrm{Sun}/h]$")
ax.set_ylabel(r"$\mathrm{Rate}[\mathrm{yr}^{-1}\mathrm{Gpc}^{-3}]$")


ax.axhline([1.], ls = "--", c = "gray")
ax.axhline([.1], ls = "--", c = "gray")
ax.axhline([.01], ls = "--", c = "gray")
ax.axhline([.0001], ls = "--", c = "gray")

ax.axhline([1e-5], ls = "--", c = "gray")

FigureCanvasNbAgg()

In [127]:
fig = plt.figure()
ax = fig.gca()

ax.plot(M_halo_lst.value, halo_mass_function, c = "k")



ax.set_xscale("log")
ax.set_yscale("log")



ax.set_xlabel(r"$M_\mathrm{c}[M_\mathrm{Sun}/h]$")
ax.set_ylabel(r"$\mathrm{d}n/\mathrm{d}ln(M)$")

ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])




FigureCanvasNbAgg()

(700.0, 700000000000000.0)

In [128]:
fig = plt.figure()
ax = fig.gca()

ax.plot(M_halo_lst.value, (1./0.7**3)*PBHPBH_Rate_per_halo.value*1e9*halo_mass_function, c = "k")
ax.plot(M_halo_lst.value, (1./0.7**3)*PBHNS_Rate_per_halo.value*1e9*halo_mass_function, c = "red")


#

ax.set_xscale("log")
ax.set_yscale("log")



ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])
#ax.set_ylim([1e-8, 1e0])


ax.set_xlabel(r"$M_\mathrm{h}[M_\mathrm{Sun}/h]$")
ax.set_ylabel(r"$\frac{\mathrm{d}n}{\mathrm{d}\ln M}\mathcal{R}[\mathrm{yr}^{-1}\mathrm{Gpc}^{-3}]$")





FigureCanvasNbAgg()

Text(0, 0.5, '$\\frac{\\mathrm{d}n}{\\mathrm{d}\\ln M}\\mathcal{R}[\\mathrm{yr}^{-1}\\mathrm{Gpc}^{-3}]$')

In [132]:
M_halo_lst = np.geomspace(1e3, 1e15, 1000)*(0.7)*u.Msun # Halo masses in M_sun/h units
M_c_lst = np.geomspace(1e3, 1e14, 10) 

m_1 = 30.*u.Msun
m_2 = 30.*u.Msun
M = m_1 + m_2
eta = m_1*m_2/M**2
binary_mass_params = {"m_1":m_1, "m_2":m_2, "M":M, "eta":eta}


############################################################################################
R_ns_over_R_NFW = 0.1
NFW_halo = halo(M_halo_lst, R_ns_over_R_NFW)
merger_rates = rates(NFW_halo, binary_mass_params)


halo_properties = NFW_halo.halo_properties()
rates_per_halo = merger_rates.rates_per_halo()
integarted_rates = merger_rates.integrated_rates(M_c_lst)

PBHPBH_Rate_per_halo_1 = rates_per_halo["PBHPBH_Rate_per_halo"]
PBHNS_Rate_per_halo_1 = rates_per_halo["PBHNS_Rate_per_halo"]
PBHPBH_rate_lst_1 = integarted_rates["PBHPBH_rate_lst"]
PBHNS_rate_lst_1 = integarted_rates["PBHNS_rate_lst"]



############################################################################################
R_ns_over_R_NFW = 0.01
NFW_halo = halo(M_halo_lst, R_ns_over_R_NFW)
merger_rates = rates(NFW_halo, binary_mass_params)


halo_properties = NFW_halo.halo_properties()
rates_per_halo = merger_rates.rates_per_halo()
integarted_rates = merger_rates.integrated_rates(M_c_lst)

PBHPBH_Rate_per_halo_2 = rates_per_halo["PBHPBH_Rate_per_halo"]
PBHNS_Rate_per_halo_2 = rates_per_halo["PBHNS_Rate_per_halo"]
PBHPBH_rate_lst_2 = integarted_rates["PBHPBH_rate_lst"]
PBHNS_rate_lst_2 = integarted_rates["PBHNS_rate_lst"]
############################################################################################








/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:130: RuntimeWarning: overflow encountered in exp


In [134]:

fig = plt.figure()
ax = fig.gca()



ax.plot(M_halo_lst, PBHPBH_Rate_per_halo_1, label = r"PBH-PBH, $f_\mathrm{PBH} = 1$", c = "red", ls = "-", zorder = 3)
ax.plot(M_halo_lst, 1e-6*PBHPBH_Rate_per_halo_1, label = r"PBH-PBH, $f_\mathrm{PBH} = 10^{-3}$", c = "red", ls = "--", zorder = 3)


ax.plot(M_halo_lst, PBHNS_Rate_per_halo_1, label = r"PBH-NS, $f_\mathrm{PBH} = 1$", c = "k", ls = "-")
ax.plot(M_halo_lst, PBHNS_Rate_per_halo_2, c = "k", ls = "-")
ax.plot(M_halo_lst, 1e-3*PBHNS_Rate_per_halo_1, label = r"PBH-NS, $f_\mathrm{PBH} = 10^{-3}$", c = "k", ls = "--")
ax.plot(M_halo_lst, 1e-3*PBHNS_Rate_per_halo_2, c = "k", ls = "--")

#ax.fill_between(M_halo_lst, PBHNS_Rate_per_halo_1, PBHNS_Rate_per_halo_2, color = "darkred", alpha = 0.3)
#ax.fill_between(M_halo_lst, 1e-3*PBHNS_Rate_per_halo_1, 1e-3*PBHNS_Rate_per_halo_2, color = "darkred", alpha = 0.3)

ratio = (PBHNS_Rate_per_halo_2/PBHNS_Rate_per_halo_1)[0]
gradient_resolution = 100
for indx, scaling in enumerate(np.geomspace(1.1, ratio, gradient_resolution)):
    tranparency = 0.6*(1. - indx/gradient_resolution)

    #print(indx, tranparency)
    ax.plot(M_halo_lst, scaling*PBHNS_Rate_per_halo_1, c = "k", ls = "-", lw = 0.365, alpha = tranparency)
    ax.plot(M_halo_lst, scaling*1e-3*PBHNS_Rate_per_halo_1, c = "k", ls = "-", lw = 0.365, alpha = tranparency)


ax.set_xscale("log")
ax.set_yscale("log")



ax.legend(ncol = 1)

ax.set_xlim([min(M_halo_lst.value), max(M_halo_lst.value)])
ax.set_ylim([1e-26, 1e-11])

plt.axvspan(min(M_halo_lst.value), 1e6, color="lightblue", alpha=0.5, zorder = 1)

ax.set_xlabel(r"$M_\mathrm{h}[M_\mathrm{Sun}/h]$")
ax.set_ylabel(r"Merger rate per halo $\mathcal{R}[1/\mathrm{yr}]$")


plt.savefig("Figs/rates_per_halo.pdf")


FigureCanvasNbAgg()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:44: UserWarning: Creating legend with loc="best" can be slow with large amounts of data.


In [135]:
fig = plt.figure()
ax = fig.gca()


ax.plot(M_c_lst, PBHNS_rate_lst_1, label = r"PBH-NS, $f_\mathrm{PBH} = 1$", c = "k", ls = "-")
ax.plot(M_c_lst, PBHNS_rate_lst_2, c = "k", ls = "-")
ax.plot(M_c_lst, 1e-3*PBHNS_rate_lst_1, label = r"PBH-NS, $f_\mathrm{PBH} = 10^{-3}$", c = "k", ls = "--")
ax.plot(M_c_lst, 1e-3*PBHNS_rate_lst_2, c = "k", ls = "--")

ax.scatter(M_c_lst, PBHNS_rate_lst_1, color = "k")
ax.scatter(M_c_lst, PBHNS_rate_lst_2, color = "k")
ax.scatter(M_c_lst, 1e-3*PBHNS_rate_lst_1, color = "k")
ax.scatter(M_c_lst, 1e-3*PBHNS_rate_lst_2, color = "k")


ratio = (PBHNS_rate_lst_2/PBHNS_rate_lst_1)[0]
gradient_resolution = 100
for indx, scaling in enumerate(np.geomspace(1.1, ratio, gradient_resolution)):
    tranparency = 0.6*(1. - indx/gradient_resolution)

    ax.plot(M_c_lst, scaling*PBHNS_rate_lst_1, c = "k", ls = "-", lw = 0.365, alpha = tranparency)
    ax.plot(M_c_lst, scaling*1e-3*PBHNS_rate_lst_1, c = "k", ls = "-", lw = 0.365, alpha = tranparency)




ax.plot(M_c_lst, PBHPBH_rate_lst_1, label = r"PBH-PBH, $f_\mathrm{PBH} = 1$", c = "red", ls = "-", zorder = 3)
ax.scatter(M_c_lst, PBHPBH_rate_lst_1, color = "red", zorder = 3)
ax.plot(M_c_lst, 1e-6*PBHPBH_rate_lst_1, label = r"PBH-PBH, $f_\mathrm{PBH} = 10^{-3}$", c = "red", ls = "--", zorder = 3)
ax.scatter(M_c_lst, 1e-6*PBHPBH_rate_lst_1, color = "red", zorder = 3)

ax.set_xscale("log")
ax.set_yscale("log")



#ax.legend(ncol = 1)

ax.set_xlim([min(M_halo_lst.value), 1.2*max(M_c_lst)])
#ax.set_ylim([1e-26, 1e-11])

plt.axvspan(min(M_halo_lst.value), 1e6, color="lightblue", alpha=0.5, zorder = 1)

ax.set_xlabel(r"$M_\mathrm{c}[M_\mathrm{Sun}/h]$")
ax.set_ylabel(r"$\mathrm{Rate}[\mathrm{yr}^{-1}\mathrm{Gpc}^{-3}]$")

ax.axhline([1.], ls = "--", c = "gray")

plt.savefig("Figs/total_rates.pdf")

FigureCanvasNbAgg()

In [ ]:
f_PBH = 1e-4

velocity_amplification_factor = (f_PBH**(1./4.))**(-11./7.)
volume_amplification_factor = f_PBH**(-1./2.)



print("Velocity enhancement factor = {:.2e} \n".format(velocity_amplification_factor), 
      "Volume enhancement factor = {:.2e} \n".format(volume_amplification_factor), 
      "Total enhancement factor = {:.2e}".format(velocity_amplification_factor*volume_amplification_factor))


In [ ]:

factor_lst = np.geomspace(1e-4, 1., 10)
#

fig = plt.figure()
ax = fig.gca()


#for indx, val in enumerate(factor_lst):

#    x_lst_test = np.geomspace(1e-5, val*C_lst, 10000).T
    
#    R_C_lst = val*R_NFW_0_lst
#    R_ns_lst_test = R_NFW_0_lst

#    density_integral = np.trapz(x_lst_test**2*NFW_tilde(x_lst_test)*NS_tilde(x_lst_test, np.array([R_C_lst, R_ns_lst_test])), x_lst_test, axis = 1)

#    ax.plot(M_halo_lst.value, density_integral, c = "black")

halo_index = 900
x_lst_test = np.geomspace(1e-10, factor_lst*C_lst[halo_index], 100000).T    
R_C_lst = np.ones(len(factor_lst))*R_NFW_0_lst[halo_index]


R_ns_test = np.ones(len(factor_lst))*R_NFW_0_lst[halo_index]
density_integral = np.trapz(x_lst_test**2*NFW_tilde(x_lst_test)*NS_tilde(x_lst_test, np.array([R_C_lst, R_ns_test])), x_lst_test, axis = 1)

ax.plot(factor_lst, density_integral/density_integral[-1], c = "black")
ax.scatter(factor_lst, density_integral/density_integral[-1], c = "black")


R_ns_test = 1e-1*np.ones(len(factor_lst))*R_NFW_0_lst[halo_index]
density_integral = np.trapz(x_lst_test**2*NFW_tilde(x_lst_test)*NS_tilde(x_lst_test, np.array([R_C_lst, R_ns_test])), x_lst_test, axis = 1)

ax.plot(factor_lst, density_integral/density_integral[-1], c = "red")
ax.scatter(factor_lst, density_integral/density_integral[-1], c = "red")



R_ns_test = 1e-2*np.ones(len(factor_lst))*R_NFW_0_lst[halo_index]
density_integral = np.trapz(x_lst_test**2*NFW_tilde(x_lst_test)*NS_tilde(x_lst_test, np.array([R_C_lst, R_ns_test])), x_lst_test, axis = 1)

ax.plot(factor_lst, density_integral/density_integral[-1], c = "green")
ax.scatter(factor_lst, density_integral/density_integral[-1], c = "green")


ax.plot(factor_lst, 1e2*factor_lst**2, c = "gray")

ax.set_xscale("log")
ax.set_yscale("log")








In [ ]:
H_0 = (cosmo.H0)*u.km/u.Mpc/u.s

R_h_cube_prefac = 2.*const.G*1e8*const.M_sun/H_0**2/0.3/20**3
R_h_prefac = (R_h_cube_prefac.to(u.pc**3))**(1/3)


print(R_h_cube_prefac.to(u.pc**3))
print(R_h_prefac)


velo_sqr_prefac = 2.*const.G*1e8*const.M_sun/const.c**2/R_h_prefac
velo_prefac = np.sqrt(velo_sqr_prefac)

print(velo_sqr_prefac.to(u.m/u.m), velo_prefac.to(u.m/u.m))


timescale_prefac = (H_0/const.c)*(const.c**4/const.G**2)*(1./9.)*(1./1e8/const.M_sun)**2*R_h_prefac**3*velo_prefac**3

print(timescale_prefac.to(u.s/u.s))

In [ ]:
m_PBH = 1.*const.M_sun
m_SMBH = 1e6*const.M_sun

mu = m_PBH*m_SMBH/(m_PBH + m_SMBH)
M = m_PBH + m_SMBH

C = -(96.*2./15.)*(const.G**3/const.c**5)*(1./H_0)*M**2*mu

1./4./C.to(u.pc**4)



In [ ]:
def f_elliptic(ecc):
    return (1. + 73.*ecc**2/24. + 37.*ecc**4/96.)*(1. - ecc**2)**(-7./2.)

one_minus_ecc_lst = np.geomspace(1e-9, 1., 1000)
f_orbit_lst = f_elliptic(1. - one_minus_ecc_lst)

fig = plt.figure()
ax = fig.gca()



ax.plot(one_minus_ecc_lst, f_orbit_lst)

ax.set_yscale("log")
ax.set_xscale("log")

ax.set_xlabel(r"$1 - e$")
ax.set_ylabel(r"$f(e)$")


In [ ]:



class Person:
  def __init__(self, fname, lname):
    self.firstname = fname
    self.lastname = lname

  def printname(self):
    print(self.firstname, self.lastname)
    
class Student(Person):
  pass

#Use the Person class to create an object, and then execute the printname method:

x = Person("John", "Doe")
x.printname()


x = Student("Mike", "Olsen")
x.printname()

x.firstname

In [ ]:


y_prime = np.array([0.142, 0.135, 0.122, 0.103])
x_prime = np.array([0.6, 0.8, 1.0, 1.2])

alpha_gamma_interpolation_function = interpolate.interp1d(x, y)

fig = plt.figure()
ax = fig.gca()

ax.scatter(x, y)
ax.scatter(x_prime, y_prime)

x_interp = np.geomspace(1., 2., 100)
ax.scatter(x_interp, alpha_gamma_interpolation_function(x_interp), c = "gray")